In [1]:

from sqlalchemy import create_engine, text

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import MoveFileTool, format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage, FunctionMessage
from langchain.tools import MoveFileTool, format_tool_to_openai_function
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI


import pandas as pd
import numpy





/Users/juliomiranda/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import pandas as pd
# Load environment variables from the .env file
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key, temperature=0)
llm = model

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) 

/Users/juliomiranda/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
TEST_MAIL = 'admin@platform.com'

## Token

In [6]:
import requests
base_url = 'https://dashboard.colorines.paitesting.com/api'

# Set the URL for the token API
url_token = base_url+"/v1/token"

# Set the data for the POST request
data = {
    "email": "sale@platform.com",
    "password": os.getenv('CONSULTANT_PASSWORD'),
    "device_name": "colorina_2",
    "role": "consultant"
}

# Send a POST request
response = requests.post(url_token, json=data)

#print(response)

# Extract the token from the response
token = response.json().get('data').get('token')

#print("Token:", token)
token_str = f"auth token: {token}"

## Components

In [ ]:
import json
import requests

def GetProperties(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/properties"
    params = {'include': 'step,project,development', 'filter[status]': 'available'} 
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the response contains any content
        if response.content:
            data = response.json()
        else:
            return json.dumps({"error": ["Empty response from the server"]})
        
    except requests.exceptions.RequestException as e:
        return json.dumps({"error": [str(e)]})
    except ValueError as ve:
        return json.dumps({"error": ["Error parsing JSON response"]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            #ids = [item['id'] for item in data['data']]
            return data
        else:
            return json.dumps({"error": ["'data' key does not contain a list"]})
    else:
        return json.dumps({"error": ["'data' key not found in the response"]})

GetProperties('123')

In [17]:
import json
import requests

def GetSales(token: str, status: str = None, customer_id: int = None, str_date: str = None, end_date: str = None):
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"  # Ensures the server knows you expect JSON
    }
    url = "https://dashboard.colorines.paitesting.com/api/v1/sales"
    params = {}
    if status:
        params.update({"filter[status]": status})
        
    if customer_id:
        params.update({"customer_id": customer_id})
        
    if str_date and end_date:
        two_dates = str_date + ',' + end_date
        params.update({"filter[created_at_between]": two_dates})
        
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the response contains any content
        if response.content:
            data = response.json()
        else:
            return json.dumps({"error": ["Empty response from the server"]})
        
    except requests.exceptions.RequestException as e:
        return json.dumps({"error": [str(e)]})
    except ValueError as ve:
        return json.dumps({"error": ["Error parsing JSON response"]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return json.dumps(data['data'])
        else:
            return json.dumps({"error": ["'data' key does not contain a list"]})
    else:
        return json.dumps({"error": ["'data' key not found in the response"]})



GetSales(token)

'[{"property": {"id": 902, "custom_id": "p805", "alias": "FA-05-805", "square_foot": {"default_value": 350, "with_decimals": "350.00", "without_decimals": "350"}, "status": {"name": "Apartado", "short_code": "booked", "is_available": false}, "prices": [{"type": "property", "amount": 1300, "amount_with_decimals": "1,300.00", "amount_without_decimals": "1300", "total_amount": 455000, "total_amount_with_decimals": "455,000.00", "total_amount_without_decimals": "455000", "currency_id": 1, "currency_symbol": "$", "currency_code": "MXN"}, {"type": "reserve", "amount": 3500, "amount_with_decimals": "3,500.00", "amount_without_decimals": "3500", "total_amount": 3500, "total_amount_with_decimals": "3,500.00", "total_amount_without_decimals": "3500", "currency_id": 1, "currency_symbol": "$", "currency_code": "MXN"}, {"type": "property", "amount": 75, "amount_with_decimals": "75.00", "amount_without_decimals": "75", "total_amount": 26250, "total_amount_with_decimals": "26,250.00", "total_amount_w

In [7]:
#TODO: API need to add a filter by 'is_active'


    
#PREPROCESS projects
def PreprocessProjects(df):
    df['development_name'] = df['development'].apply(lambda x: x.get('name'))
    
    def filter_active_steps(steps):
        return [step['step_number'] for step in steps if step['is_active']]

    # Applying the function to create the new column
    df['active_steps'] = df['steps'].apply(filter_active_steps)

    return df

#PREPROCESS Properties
def PreprocessProperties(df):
    df['project_name'] = df['project'].apply(lambda x: x.get('name'))
    df['development_name'] = df['development'].apply(lambda x: x.get('name'))
    df['step_number'] = df['step'].apply(lambda x: x.get('step_number')) #numero de etapa

    def extract_total_amount(prices, currency_code):
        total_amount = None  # Initialize to None or any default value as needed
        for entry in prices:
            if entry['type'] == 'property' and entry['currency_code'] == currency_code:
                total_amount = entry['total_amount']
                break  # Exit loop once found
        return total_amount

    # Extracting total_amount_mxn and total_amount_usd
    df['total_amount_mxn'] = df['prices'].apply(lambda x: extract_total_amount(x, 'MXN'))
    df['total_amount_usd'] = df['prices'].apply(lambda x: extract_total_amount(x, 'USD'))

    return df


# Example usage

#df_properties = GetProperties(token)

def GetProjects(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/projects"
    params = {'include': 'steps,development', 'is_active': 'True'} 
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

def GetDevelopments(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/developments"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

# Example usage

#df_devs = GetDevelopments(token)



### Sales

In [8]:
def GetSales(token: str, status: str = None, customer_id: int = None, str_date: str = None, end_date: str = None):
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"  # Ensures the server knows you expect JSON
    }
    url = "https://dashboard.colorines.paitesting.com/api/v1/sales"
    params={}
    if status:
        params.update({"filter[status]": status})
        
    if customer_id:
        params.update({"customer_id": customer_id})
        
    if str_date and end_date:
        two_dates = str_date+','+end_date
        params.update({"filter[created_at_between]": two_dates})
        
    try:
        response = requests.get(url, headers=headers, params=params)
        #print("Status Code:", response.status_code)  # Log the status code
        #print("Response Body:", response.text)   
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the response contains any content
        if response.content:
            data = response.json()
        else:
            return pd.DataFrame({"error": ["Empty response from the server"]})
        
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    except ValueError as ve:
        return pd.DataFrame({"error": ["Error parsing JSON response"]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

def PreprocessSales(df):
    df['property_name'] = df['property'].apply(lambda x: x.get('alias'))

    return df

### Customers

In [9]:
def GetCustomers(token: str):
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"  # Ensures the server knows you expect JSON
    }
    url = "https://dashboard.colorines.paitesting.com/api/v1/customers"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the response contains any content
        if response.content:
            data = response.json()
        else:
            return pd.DataFrame({"error": ["Empty response from the server"]})
        
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    except ValueError as ve:
        return pd.DataFrame({"error": ["Error parsing JSON response"]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

def GetCustomerID(df, email):
    df_filtered = df[df['email'] == email]
    if (len(df_filtered)>0):
        return df_filtered.iloc[0].id
    else:
        return None

## Clean functions

In [10]:
def CleanProjects(df):
    df_clean = df[['name', 'development_name', 'active_steps']]
    df_clean = df_clean.rename(columns={'name': 'nombre_proyecto', 'development_name': 'nombre_desarrollo', 'active_steps': 'etapa'})
    return df_clean.to_json(orient='records', force_ascii=False)

def CleanProperties(df):
    df_clean = df[['alias', 'status', 'project_name', 'development_name', 'step_number', 'total_amount_mxn', 'total_amount_usd']]
    
    df_clean = df_clean.rename(columns={'alias': 'nombre_propiedad',
                                        'project_name': 'nombre_proyecto',
                                        'development_name': 'nombre_desarrollo',
                                        'step_number': 'etapa',
                                        'total_amount_mxn': 'precio_MXN',
                                        'total_amount_usd': 'precio_USD'
                                       })
    return df_clean.to_json(orient='records', force_ascii=False)

def CleanSalesByCustomer(df):
    df_clean = df[['property_name', 'currency', 'reserve_amount', 'property_amount', 'property_discount_amount', 'down_payment_amount', 'down_payment_percent', 'number_installments', 'installments_amount', 'status', 'amount_total', 'amount_paid', 'amount_remaining', 'next_payment_date', 'advance_percent', 'overdue_installments_count', 'overdue_installments_amount']]
    return df_clean.to_json(orient='records', force_ascii=False)

## Tools

In [11]:
@tool
def get_active_properties(token: str) -> str:
    """
    Uses the token to fetch the active properties from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    properties_df = GetProperties(token)
    properties_df = properties_df[properties_df['is_active'] == True]
    

    response = requests.get(url, headers=headers, params=params)
    # Check if the response was successful
    if response.status_code == 200:
        try:
            return json.dumps(response.json())  # Convert JSON response to string
        except ValueError:  # Catch JSON errors
            return json.dumps({"error": "Error parsing JSON"})
    else:
        return json.dumps({"error": f"Received status code {response.status_code}"})



@tool
def get_last_sale(token: str) -> str:
    """
    Uses the token to fetch the last sale from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/sales"
    params = {"per_page": 1, "sort": "-created_at"}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        # Check if the response was successful
        if response.status_code == 200:
            try:
                return json.dumps(response.json())  # Convert JSON response to string
            except ValueError:  # Catch JSON errors
                return json.dumps({"error": "Error parsing JSON"})
        else:
            return json.dumps({"error": f"Received status code {response.status_code}"})
    except requests.exceptions.RequestException as e:
        return json.dumps({"error": str(e)})



### Properties, developments and Steps

In [12]:
@tool
def get_active_projects(token: str) -> str:   
    """
    Uses the token to fetch the active (available) projects from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    df = GetProjects(token)
    pre_df = PreprocessProjects(df)
    return CleanProjects(pre_df)
    
@tool
def get_active_properties(token: str)-> str:
    """
    Uses the token to fetch the active (available) properties from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    
    df = GetProperties(token)
    pre_df = PreprocessProperties(df)
    return CleanProperties(pre_df)



### Sales

In [13]:
@tool
def count_sales_by_status(token: str, status: str)-> int:
    """
    Count the number of sales the user has with a specific status.
    
    
    Args:
        token (str): The authorization token required to access the API.
        status (str): The status to filter the API request. Acceptable status by the API: reserved, documents, contract, active, canceled, overdue, finalized, deeded.

    Returns:
        int: The number of sales with the requested status.
    """
    
    df = GetSales(token=token, status=status)
    count = len(df)
    return count

@tool
def get_sales_by_customer(token: str, email: str) -> str:
    """
    Retrieves and returns a list of sales associated with a customer identified by their email.

    This function performs the following steps:
    1. Uses the provided email to search for the customer ID.
    2. Retrieves the sales attached to that customer ID.
    3. Returns relevant sales information in JSON format.

    Args:
        token (str): The authorization token required to access the API.
        email (str): The email address used to identify the customer.

    Returns:
        str: A JSON response containing information about sales made to the customer.
             The JSON includes details about each sale that will be listed to the user by the agent.
    """
    # Function implementation here
    customers = GetCustomers(token)
    customer_id = GetCustomerID(customers, email=email)
    if customer_id:
        df = GetSales(token=token, customer_id=customer_id)
        df = PreprocessSales(df)
        return CleanSalesByCustomer(df)
    else:
        return 'No se encontró ningún cliente con ese correo electrónico, pregunta al cliente si el correo es correcto'




In [14]:
def call_json_output_parser():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Extract information from the Colorines API using the {token}.\nFormatting Instructions: {format_instructions}"),
        ("human", "{input}")
    ])

    class Data(BaseModel):
        data: str = Field(description="JSON with the response containing all the relevant information for the user")
        
    parser = JsonOutputParser(pydantic_object=Data)

    chain = prompt | model | parser
    
    return chain.invoke({
        "phrase": "The ingredients for a Margherita pizza are tomatoes, onions, cheese, basil",
        "format_instructions": parser.get_format_instructions()
    })

#### Sales remake

In [15]:
# Count sales
class CountSalesInput(BaseModel):
    """Input to grant access to Colorines API."""

    token: str = Field(..., description="token to authenticate the user to consume the API")
    status: str = Field(..., description="Status to filter the sales to retrieve from the API. Acceptable status by the API: reserved, documents, contract, active, canceled, overdue, finalized, deeded")
    
class CountSalesTool(BaseTool):
    name = "count_sales_by_status"
    description = "Count the number of sales the user has with a specific status. You must input the auth token"

    def _run(self, token: str, status: str):
        df = GetSales(token=token, status=status)
        count = len(df)
        return json.dumps({"count": count})  # Directly return JSON string


    def _arun(self, token: str, status: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = CountSalesInput
    
#list sales by email
# Input schema for fetching sales by customer
class GetSalesByCustomerInput(BaseModel):
    """Input to fetch sales by customer."""

    token: str = Field(..., description="Authorization token to access the API.")
    email: str = Field(..., description="Email address of the customer.")

# Tool class to retrieve sales by customer
class GetSalesByCustomerTool(BaseTool):
    name = "get_sales_by_customer"
    description = "Retrieve sales associated with a customer identified by email."

    def _run(self, token: str, email: str):
        """
        Retrieves and returns sales associated with a customer identified by email.

        Args:
            token (str): Authorization token to access the API.
            email (str): Email address of the customer.

        Returns:
            str: JSON response containing information about sales made to the customer.
        """
        # Replace these with your actual functions to fetch data
        customers = GetCustomers(token)
        customer_id = GetCustomerID(customers, email=email)
        
        if customer_id:
            df = GetSales(token=token, customer_id=customer_id)
            df = PreprocessSales(df)
            return json.dumps(CleanSalesByCustomer(df))
        else:
            return json.dumps({
                "message": "No se encontró ningún cliente con ese correo electrónico. "
                           "Verifica el correo proporcionado e intenta nuevamente."
            })

    def _arun(self, token: str, email: str):
        raise NotImplementedError("This tool does not support async.")

    args_schema: Optional[Type[BaseModel]] = GetSalesByCustomerInput
    

from pydantic import BaseModel, Field
from typing import Optional, Type

class GetSalesBetweenDatesInput(BaseModel):
    """
    Defines the input schema for fetching sales between specified dates.
    """
    token: str = Field(..., description="Authorization token required to access the API.")
    str_date: str = Field(..., description="Start date to search sales created in between. Format must be: 'yyyy-mm-dd'")
    end_date: str = Field(..., description="End date to search sales created in between. Format must be: 'yyyy-mm-dd'")

class GetSalesBetweenDatesTool(BaseTool):
    name = "get_sales_between_dates"  
    description = "Retrieve sales in a range of two dates. If only one date is provided, the same date is used for both start and end dates."

    def _run(self, token: str, str_date: str, end_date: str):
        """
        Executes a query to retrieve sales that were created between two specified dates.

        Args:
            token (str): Authorization token to access the API.
            str_date (str): Start date to begin the search for sales.
            end_date (str): End date to end the search for sales.

        Returns:
            str: JSON response containing information about sales in the specified date range.
        """
        if str_date and end_date:
            df = GetSales(token=token, str_date=str_date, end_date=end_date)
            df = PreprocessSales(df)
            return json.dumps(CleanSalesByCustomer(df))
        else:
            return json.dumps({
                "message": "Both start and end dates must be provided."
            })

    def _arun(self, token: str, str_date: str, end_date: str):
        raise NotImplementedError("This tool does not support asynchronous execution.")

    args_schema: Optional[Type[BaseModel]] = GetSalesBetweenDatesInput




In [16]:
tools = [CountSalesTool(), GetSalesByCustomerTool(), GetSalesBetweenDatesTool()]

## Half agent use, just to select Tools

In [63]:
import platform
print(platform.python_version())

3.10.9


In [18]:
#ChatGpt model to select tools
model = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

#The tools classes
tool_names = [CountSalesTool().name, GetSalesByCustomerTool().name, GetSalesBetweenDatesTool().name]
tools = [CountSalesTool(), GetSalesByCustomerTool(), GetSalesBetweenDatesTool()]

#tool_names = [MultiplyTool().name, SumTool().name]
#tools = [MultiplySalesTool(), SumSalesTool()]

#Format Tools for being usable by the model
functions = [format_tool_to_openai_function(t) for t in tools]

#Token append to input string
token_str = f"mi auth token: {token}"

#Uses the model to predict which tool use, but we return the tool response directly
def smart_tool_selector(input_str, token_str, model, functions, tool_names):
    """
    Uses a machine learning model to predict which tool to use based on input,
    then directly returns the response from the selected tool.

    Args:
    - input_str (str): The input string to be processed.
    - token_str (str): The token string to be included in the input.
    - model: The machine learning model used to predict the tool.
    - functions (list): List of functions or tools available for selection.

    Returns:
    - tool_result: The result returned by the selected tool.
    - _args (dict): Arguments parsed from the AI model's function call prediction.
    - ai_message: Message object containing details of the AI model's prediction.
    """
    # Prepare input data for the model prediction
    input_data = {
        "input": f"{input_str} {token_str}"
    }
    
    # Predict the tool to use based on input using the machine learning model
    ai_message = model.predict_messages([HumanMessage(content=input_data['input'])], functions=functions)
    
    # Extract arguments for the tool from the AI model's function call prediction
    _args = json.loads(ai_message.additional_kwargs['function_call'].get('arguments'))
    
    print(_args)
    
    tool_name = ai_message.additional_kwargs['function_call']['name']
    tool_index = tool_names.index(tool_name)
    # Execute the selected tool and capture its result
    tool_result = tools[tool_index](_args)  # Pass _args as keyword arguments
    
    # Return the tool's result, parsed arguments, and AI message
    return tool_result, _args, ai_message



/Users/juliomiranda/anaconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [19]:
input_str = 'muestrame las ventas entre el 7 de enero del 2024 y primero de junio del 2024?'
result, args, msg = smart_tool_selector(input_str, token_str, model, functions, tool_names)
print(msg)
print(args)
print(result)


/Users/juliomiranda/anaconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'token': '2391|yXT4bBKLz8cwUBTQXnA9MBhQWDmSIbFd6yAODN2j1738ed60', 'str_date': '2024-01-07', 'end_date': '2024-06-01'}


/Users/juliomiranda/anaconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


KeyError: 'property'

In [20]:
tool_names

['count_sales_by_status', 'get_sales_by_customer', 'get_sales_between_dates']

In [21]:
input_str = 'cuanto es 2 + 3?'
result, args, msg = smart_tool_selector(input_str, model, functions, tool_names)
print(msg)
print(args)
print(result)

TypeError: smart_tool_selector() missing 1 required positional argument: 'tool_names'

In [22]:
input_str='muéstrame las ventas del cliente admin@platform.com'
result, args, msg = smart_tool_selector(input_str, token_str, model, functions, tool_names)
result

{'token': '2391|yXT4bBKLz8cwUBTQXnA9MBhQWDmSIbFd6yAODN2j1738ed60', 'email': 'admin@platform.com'}


KeyError: 'property'

In [ ]:
TEST_MAIL

## Stable old agent

In [57]:
tools

[CountSalesTool(), GetSalesByCustomerTool(), GetSalesBetweenDatesTool()]

Problem: Langchain doc is wrong: https://api.python.langchain.com/en/latest/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html#langchain.agents.tool_calling_agent.base.create_tool_calling_agent

In [52]:
    
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant who will use the {token} to use the tools to bring information to the user"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])


# Add the `get_sales` function to the list of tools
tools_api = [get_last_sale, get_active_projects, count_sales_by_status, get_sales_by_customer] 

class Result(BaseModel):
    data: str = Field(description="The API response returned by a tool")

parser = JsonOutputParser(pydantic_object=Result)


# Create the agent message_formatter
agent = create_tool_calling_agent(llm=llm, tools=tools_api, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_api, verbose=True)

# Example usage:
# token = 'your_actual_token_here'
# base_url = 'https://api.yourdomain.com'


In [53]:
TEST_MAIL

'admin@platform.com'

In [58]:
input_str = "cuantas ventas en entre el 1 de junio del 2024 y el 10 de julio del 2024?"

input_data = {
        "input": f"{input_str} {token_str}"
}

inp = f"{input_str} {token_str}"

input_data_old = {
        "input": input_str,
        "token": token
}

In [59]:


# Invoke the agent
agent_executor.invoke(input_data_old)



> Entering new AgentExecutor chain...

Invoking: `get_active_projects` with `{'token': '2391|yXT4bBKLz8cwUBTQXnA9MBhQWDmSIbFd6yAODN2j1738ed60'}`


[{"nombre_proyecto":"Tres Raíces","nombre_desarrollo":"Yucatán","etapa":[4]}]
Invoking: `get_last_sale` with `{'token': '2391|yXT4bBKLz8cwUBTQXnA9MBhQWDmSIbFd6yAODN2j1738ed60'}`


{"error": "Error parsing JSON"}Lo siento, parece que hubo un error al intentar obtener la información sobre la última venta. ¿Hay algo más en lo que pueda ayudarte?

> Finished chain.


{'input': 'cuantas ventas en entre el 1 de junio del 2024 y el 10 de julio del 2024?',
 'token': '2391|yXT4bBKLz8cwUBTQXnA9MBhQWDmSIbFd6yAODN2j1738ed60',
 'output': 'Lo siento, parece que hubo un error al intentar obtener la información sobre la última venta. ¿Hay algo más en lo que pueda ayudarte?'}

## Demo from zero

In [27]:
import requests
from langchain.chat_models import ChatOpenAI
from langchain.chains import SimpleChain

# Configuración del modelo
model = ChatOpenAI(api_key='your_openai_api_key')

# Definición de herramientas
def query_api(endpoint, params):
    response = requests.get(endpoint, params=params)
    return response.json()

tools = {
    'query_api': GetSalesByCustomerTool
}

# Creación del prompt
prompt = """
Actúa como un asistente inteligente que consulta la API de nuestro cliente. Cuando recibas una solicitud, utiliza las herramientas disponibles para obtener la información necesaria y devuélvela en formato JSON.

Solicitud: {user_input}
"""

# Configuración del parser
def parse_response(response):
    return {
        'status': response.get('status'),
        'data': response.get('data')
    }

# Integración de LangChain
chain = SimpleChain(
    model=model,
    prompt=prompt,
    tools=tools,
    parser=parse_response
)

# Uso del chatbot
user_input = "Consultar datos del cliente con ID 123"
response = chain.run(user_input)
print(response)

ImportError: cannot import name 'SimpleChain' from 'langchain.chains' (/Users/juliomiranda/anaconda3/lib/python3.10/site-packages/langchain/chains/__init__.py)

## Demo json out parser langchain doc

In [28]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

llm_with_tools = model.bind_tools(tools)




# Define your desired data structure.
class Result(BaseModel):
    data: str = Field(description="Data with the API response")


# And a query intented to prompt a language model to populate the data structure.

input_str = "Can you tell me my auth token?"

input_data = {
    "input": f"{input_str} {token_str}"
}

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Result)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

#chain.invoke({"query": input_data})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_I3zUsKh7jKOKHSRGZ1VZiUb0', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 95, 'total_tokens': 113}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d7822315-c674-483a-bee9-567e1cd386b4-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_I3zUsKh7jKOKHSRGZ1VZiUb0'}], usage_metadata={'input_tokens': 95, 'output_tokens': 18, 'total_tokens': 113})

##  Chatgpt tests

In [29]:
from langchain.tools import Tool, ToolSelector

ImportError: cannot import name 'ToolSelector' from 'langchain.tools' (/Users/juliomiranda/anaconda3/lib/python3.10/site-packages/langchain/tools/__init__.py)

In [30]:
import os
import openai
from langchain import LLMChain, Prompt, Model, JsonOutputParser
from langchain.tools import Tool

# Set your OpenAI API key
openai.api_key = 'your-openai-api-key'

# Define the prompt template
prompt_template = """
You are a helpful assistant. Answer the following question in a JSON format. If the question involves arithmetic calculations, use the provided tools to compute the answer.
Question: {question}
"""

# Create the Prompt object
prompt = Prompt(template=prompt_template)

# Define the model (ChatGPT)
model = Model(
    model_name="gpt-3.5-turbo",  # You can change this to the desired model
    max_tokens=150  # Adjust based on your needs
)

# Define the summation tool
def sum_tool(question):
    try:
        # Extract numbers from the question and calculate the sum
        numbers = [float(num) for num in question.split() if num.replace('.', '', 1).isdigit()]
        result = sum(numbers)
        return {"result": result}
    except Exception as e:
        return {"error": str(e)}

# Create a Tool object for the summation tool
sum_tool_obj = Tool(
    name="SumTool",
    description="Performs summation of numbers.",
    func=sum_tool
)

# Define the multiplication tool
def multiply_tool(question):
    try:
        # Extract numbers from the question and calculate the product
        numbers = [float(num) for num in question.split() if num.replace('.', '', 1).isdigit()]
        result = 1
        for num in numbers:
            result *= num
        return {"result": result}
    except Exception as e:
        return {"error": str(e)}

# Create a Tool object for the multiplication tool
multiply_tool_obj = Tool(
    name="MultiplyTool",
    description="Performs multiplication of numbers.",
    func=multiply_tool
)

# Define the JSON output parser
json_output_parser = JsonOutputParser()

# Create the chain with tools
chain = LLMChain(
    prompt=prompt,
    model=model,
    tools=[sum_tool_obj, multiply_tool_obj],
    output_parser=json_output_parser
)

# Define a function to run the chain
def run_chain(question):
    input_data = {"question": question}
    result = chain.run(input_data)
    return result

# Example usage
question = "What is the sum of 5 and 3?"
response = run_chain(question)
print(response)

# Another example
question = "What is the product of 5 and 3?"
response = run_chain(question)
print(response)


ImportError: cannot import name 'Model' from 'langchain' (/Users/juliomiranda/anaconda3/lib/python3.10/site-packages/langchain/__init__.py)

## Test YT LangChain Tutorial (Python) #6: Self-Reasoning Agents with Tools

Video: https://www.youtube.com/watch?v=uyAyeUcXps8

In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor

In [34]:
tools

[CountSalesTool(), GetSalesByCustomerTool(), GetSalesBetweenDatesTool()]

In [35]:
TEST_MAIL

'admin@platform.com'

In [36]:
#added this line from langchian docs
llm_with_tools = model.bind_tools(tools)

In [37]:
prompt = ChatPromptTemplate.from_messages([
        ("system", "you are a Friendly assitant named colorina"),
        ("human", "{input}"), 
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ])

agent = create_openai_functions_agent(
    llm=model,
    prompt=prompt,
    tools=tools,
)

input_str = 'cuantas ventas tengo con el cliente admin@platform.com?'

input_data = {
    "input": f"{input_str} {token_str}"
}

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(input_data)





> Entering new AgentExecutor chain...

Invoking: `get_sales_by_customer` with `{'token': '2391|yXT4bBKLz8cwUBTQXnA9MBhQWDmSIbFd6yAODN2j1738ed60', 'email': 'admin@platform.com'}`




KeyError: 'property'

In [38]:
llm_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11c9b2e60>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11c9b3850>, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'count_sales_by_status', 'description': 'Count the number of sales the user has with a specific status. You must input the auth token', 'parameters': {'type': 'object', 'properties': {'token': {'description': 'token to authenticate the user to consume the API', 'type': 'string'}, 'status': {'description': 'Status to filter the sales to retrieve from the API. Acceptable status by the API: reserved, documents, contract, active, canceled, overdue, finalized, deeded', 'type': 'string'}}, 'required': ['token', 'status']}}}, {'type': 'function', 'function': {'name': 'get_sales_by_customer', 'description': 'Retrieve sales associated with a customer identified by email.', 'paramete

In [39]:
def call_json_output_parser(llm_with_tools):
    prompt = ChatPromptTemplate.from_messages([
            ("system", "you are a Friendly assitant named colorina. If necessary, you will consult the colorines API with the tools provided to solve the user request. Use the auth token to use the tools. \nFormatting Instructions: {format_instructions}"),
            ("human", "{input}"), 
        ])

    class Result(BaseModel):
        data: str = Field(description="Data returned by the tools")
        
    parser = JsonOutputParser(pydantic_object=Result)

    chain = prompt | llm_with_tools | parser
    
    input_str = 'cuantas ventas tengo con el cliente admin@platform.com?'

    input_data = {
        "input": f"{input_str} {token_str}",
        "format_instructions": parser.get_format_instructions()
    }
    
    return chain.invoke(input_data)

In [40]:
call_json_output_parser(llm_with_tools)

OutputParserException: Invalid json output: 

In [47]:
def call_json_output_parser_v2(token_str, llm_with_tools, input_str):
    prompt = ChatPromptTemplate.from_messages([
            ("system", "you will use the tools to provide an answer. \nFormatting Instructions: {format_instructions}"),
            ("human", "{input}"), 
        ])

    class Result(BaseModel):
        data: str = Field(description="Should contain a json structure with the results of the tools")
        
    parser = JsonOutputParser(pydantic_object=Result)

    chain = prompt | llm_with_tools | parser

    input_data = {
        "input": f"{input_str} {token_str}",
        "format_instructions": parser.get_format_instructions()
    }
    
    return chain.invoke(input_data)

In [ ]:
call_json_output_parser_v2(token_str, llm_with_tools, 'cuantas ventas tengo con el cliente admin@platform.com?')

In [102]:
call_json_output_parser_v2(token_str, model, 'hola? ')

{'properties': {'data': {'title': 'Data',
   'description': 'Should contain a json structure with the results of the tools',
   'type': 'string'}},
 'required': ['data']}

### Add parser with no pydantic

In [41]:
def call_json_output_parser_v3(token_str, llm_with_tools, input_str):
    prompt = ChatPromptTemplate.from_messages([
            ("system", '''You will use the tools binded to the llm model to provide an answer. The response must be only a json with the results of the tools.
            The JSON structure may vary depending on the request response'''),
            ("human", "{input}"), 
        ])

        
    parser = JsonOutputParser()

    chain = prompt | llm_with_tools | parser

    input_data = {
        "input": f"{input_str} {token_str}",
    }
    
    return chain.invoke(input_data)

In [42]:
token

'2391|yXT4bBKLz8cwUBTQXnA9MBhQWDmSIbFd6yAODN2j1738ed60'

In [43]:
call_json_output_parser_v3(token_str, model, 'cuantas ventas tengo?')

{'error': 'Unauthorized access. Please provide a valid authentication token.'}

In [44]:
call_json_output_parser_v3(token_str, model, 'cuantas ventas tengo con el cliente admin@platform.com?')

{'tool_1': {'total_sales': 15}, 'tool_2': {'total_sales': 10}}

## Try with MasterClass

ytvideo: https://www.youtube.com/watch?v=yF9kGESAi3M

In [45]:
from langchain import hub
from langchain_community.llms import OpenAI
from langchain.agents import AgentExecutor, create_react_agent

In [46]:
tools

[CountSalesTool(), GetSalesByCustomerTool(), GetSalesBetweenDatesTool()]

In [47]:
def create_prompt_template():
    prompt = ChatPromptTemplate.from_messages([
        ("system", '''
        You are a capable assistant with access to various tools. Your tasks are:
        1. Identify when a tool is needed based on the user's input.
        2. Collect the necessary parameters required for the tool or function to work.
        3. Use the provided token for authentication to utilize the tools via an API.
        4. Always provide your response in JSON format with the results from the tools.

        Instructions for using tools:
        - When a tool is required, gather all necessary parameters.
        - Use the given token for API authentication to access the tools.
        - Ensure the response structure in JSON format varies as per the tool's output.

        Example JSON Response:
        {
            "tool_used": "count_sales_by_status",
            "parameters": {
                "token": "your_token_here",
                "status": "documents"
            },
            "results": {
                "count": 10
            }
        }

        Available tools:
        {tools}

        Tool names:
        {tool_names}
        '''),

        ("human", "{input}"),

        ("assistant", '''
        Agent scratchpad:
        {agent_scratchpad}
        ''')
    ])

    return prompt

In [48]:
#prompt = hub.pull("hwchase17/react")

prompt=create_prompt_template()
agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
    #stop_sequence=True
    
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
)



In [49]:
input_str='how many sale i have in documents status?'
input_data = {
        "input": f"{input_str}. token: {token}"
    }

In [50]:
input_data

{'input': 'how many sale i have in documents status?. token: 2391|yXT4bBKLz8cwUBTQXnA9MBhQWDmSIbFd6yAODN2j1738ed60'}

In [51]:

response = agent_executor.invoke(input_data)



> Entering new AgentExecutor chain...


KeyError: 'Input to ChatPromptTemplate is missing variables {\'\\n            "tool_used"\'}.  Expected: [\'\\n            "tool_used"\', \'agent_scratchpad\', \'input\'] Received: [\'input\', \'intermediate_steps\', \'agent_scratchpad\']'